# U.S. Medical Insurance Costs

In [2]:
import pandas as pd
import numpy as np

insurance_records = pd.read_csv('insurance.csv')
print(insurance_records.info())
insurance_records.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB
None


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [17]:
# Obtain the average insurance costs for female and male and calculate the difference
average_cost_sex = insurance_records.groupby('sex').charges.mean().reset_index()
average_cost_sex.rename(columns={'charges': 'average insurance charges'}, inplace=True)
average_cost_sex['difference between sex'] = average_cost_sex['average insurance charges'].diff()

average_cost_sex.head()

,sex,average insurance charges,difference between sex
0,female,12569.578844,NaN
1,male,13956.751178,1387.172334


In [4]:
# Find the first and third quartile and the interquartile range of insurance costs for female and male
sex_third_q = insurance_records.groupby('sex').charges.apply(lambda x: np.percentile(x, 75)).reset_index()
sex_first_q = insurance_records.groupby('sex').charges.apply(lambda x: np.percentile(x, 25)).reset_index()
sex_interquartile = (sex_third_q.charges - sex_first_q.charges).rename({0: 'female', 1: 'male'}).reset_index()

sex_third_q.head()

,sex,charges
0,female,14454.691825
1,male,18989.590250


In [5]:
sex_first_q.head()

,sex,charges
0,female,4885.1587
1,male,4619.1340


In [6]:
sex_interquartile.head()

,index,charges
0,female,9569.533125
1,male,14370.456250


In [7]:
# Obtain amount of patients from each region and the respective average insurance costs
regions = insurance_records.value_counts('region').rename('Number of patients', inplace=True).reset_index()

# Another approach to count amount of patients from each region
#regions = insurance_records.groupby('region').age.count().rename('Number of patients', inplace=True).reset_index()

average_cost_regions = insurance_records.groupby('region').charges.mean().reset_index()
average_cost_regions.rename(columns={'charges': 'Average insurance charges'}, inplace=True)

merged_regions = regions.merge(average_cost_regions)

# Find the first and third quartile and the interquartile range of insurance costs in each region
regions_first_q = insurance_records.groupby('region').charges.apply(lambda x: np.percentile(x, 25)).reset_index()
regions_third_q = insurance_records.groupby('region').charges.apply(lambda x: np.percentile(x, 75)).reset_index()
regions_interquartile = (regions_third_q.charges - regions_first_q.charges).rename({0: 'southeast', 1: 'northwest', 2: 'southwest', 3: 'northeast'}).reset_index()

merged_regions.head()

,region,Number of patients,Average insurance charges
0,southeast,364,14735.411438
1,northwest,325,12417.575374
2,southwest,325,12346.937377
3,northeast,324,13406.384516


In [8]:
regions_first_q.head()

,region,charges
0,northeast,5194.322288
1,northwest,4719.736550
2,southeast,4440.886200
3,southwest,4751.070000


In [9]:
regions_third_q.head()

,region,charges
0,northeast,16687.3641
1,northwest,14711.7438
2,southeast,19526.2869
3,southwest,13462.5200


In [10]:
regions_interquartile.head()

,index,charges
0,southeast,11493.041812
1,northwest,9992.007250
2,southwest,15085.400700
3,northeast,8711.450000


In [11]:
# Calculate average insurance costs between smoker and non-smoker
average_cost_smoker = insurance_records.groupby('smoker').charges.mean().reset_index()
average_cost_smoker.rename(columns={'charges': 'Average insurance charges'}, inplace=True)
average_cost_smoker['Difference between smoker and non-smoker'] = average_cost_smoker['Average insurance charges'].diff()

average_cost_smoker.head()

,smoker,Average insurance charges,Difference between smoker and non-smoker
0,no,8434.268298,NaN
1,yes,32050.231832,23615.963534


In [31]:
# Find the first and third quartile and the interquartile range of insurance costs of smokers and non-smokers
smoker_first_q = insurance_records.groupby('smoker').charges.apply(lambda x: np.percentile(x, 25)).reset_index()
smoker_third_q = insurance_records.groupby('smoker').charges.apply(lambda x: np.percentile(x, 75)).reset_index()
smoker_interquartile = insurance_records.groupby('smoker').charges.apply(lambda x: np.percentile(x, 50)).reset_index()

In [27]:
smoker_first_q.head()

,smoker,charges
0,no,3986.438700
1,yes,20826.244213


In [28]:
smoker_third_q.head()

,smoker,charges
0,no,11362.887050
1,yes,41019.207275


In [32]:
smoker_interquartile.head()

,smoker,charges
0,no,7345.40530
1,yes,34456.34845


In [21]:
# Add a new column of age groups and sort patients into their corresponding generation
age_bins = [17, 26, 42, 58, 68]
gen_labels = ['Gen Z', 'Millenials', 'Gen X', 'Boomers II (Generation Jones)']
insurance_records['age group'] = pd.cut(insurance_records['age'], bins=age_bins, labels=gen_labels)

# Obtain average age of patients and average insurance cost for different age groups
average_cost_age = insurance_records.groupby('age group').charges.mean().reset_index()
average_cost_age.head()

,age group,charges
0,Gen Z,8839.442951
1,Millenials,11460.191605
2,Gen X,15955.043122
3,Boomers II (Generation Jones),20824.972901


In [33]:
# Find the first and third quartile and the interquartile range of insurance costs for each age group
age_groups_first_q = insurance_records.groupby('age group').charges.apply(lambda x: np.percentile(x, 25)).reset_index()
age_groups_third_q = insurance_records.groupby('age group').charges.apply(lambda x: np.percentile(x, 75)).reset_index()
age_groups_interquartile = insurance_records.groupby('age group').charges.apply(lambda x: np.percentile(x, 50)).reset_index()

In [34]:
age_groups_first_q.head()

,age group,charges
0,Gen Z,1965.340675
1,Millenials,4766.022000
2,Gen X,9180.161412
3,Boomers II (Generation Jones),13143.600750


In [35]:
age_groups_third_q.head()

,age group,charges
0,Gen Z,14060.904812
1,Millenials,16420.494550
2,Gen X,19964.290327
3,Boomers II (Generation Jones),27875.006340


In [36]:
age_groups_interquartile.head()

,age group,charges
0,Gen Z,2731.010025
1,Millenials,6313.759000
2,Gen X,10962.570350
3,Boomers II (Generation Jones),14119.620000


In [24]:
# Obtain average age for a patient in this dataset who has at least one child
insurance_records['have children'] = np.where(insurance_records['children'] != 0, 'yes', 'no')
has_children_average_age = insurance_records.groupby('have children').age.mean().reset_index()
has_children_average_age.head()

,have children,age
0,no,38.444251
1,yes,39.780105


In [52]:
# Obtain average insurance costs for patients with and without children and calculate their difference
average_cost_children = insurance_records.groupby('have children').charges.mean().reset_index()

average_cost_children.head()

,have children,charges
0,no,12365.975602
1,yes,13949.941093


In [53]:
# Obtain different average insurance costs with respect to amount of children
average_cost_children_amount = insurance_records.groupby('children').charges.mean().reset_index()

average_cost_children_amount.head()

,children,charges
0,0,12365.975602
1,1,12731.171832
2,2,15073.563734
3,3,15355.318367
4,4,13850.656311


In [37]:
# Find the first and third quartile and the interquartile range of insurance costs for patients with and without children
have_children_first_q = insurance_records.groupby('have children').charges.apply(lambda x: np.percentile(x, 25)).reset_index()
have_children_third_q = insurance_records.groupby('have children').charges.apply(lambda x: np.percentile(x, 75)).reset_index()
have_children_interquartile = insurance_records.groupby('have children').charges.apply(lambda x: np.percentile(x, 50)).reset_index()

In [38]:
have_children_first_q.head()

,have children,charges
0,no,2734.421150
1,yes,5809.641625


In [39]:
have_children_third_q.head()

,have children,charges
0,no,14440.123825
1,yes,18232.392400


In [40]:
have_children_interquartile.head()

,have children,charges
0,no,9856.9519
1,yes,9223.8295


In [54]:
# Find the first and third quartile and the interquartile range of insurance costs with different amount of children
children_amount_first_q = insurance_records.groupby('children').charges.apply(lambda x: np.percentile(x, 25)).reset_index()
children_amount_third_q = insurance_records.groupby('children').charges.apply(lambda x: np.percentile(x, 75)).reset_index()
children_amount_interquartile = insurance_records.groupby('children').charges.apply(lambda x: np.percentile(x, 50)).reset_index()

In [55]:
children_amount_first_q.head()

,children,charges
0,0,2734.421150
1,1,4791.643175
2,2,6284.939438
3,3,6652.528800
4,4,7512.267000


In [56]:
children_amount_third_q.head()

,children,charges
0,0,14440.123825
1,1,15632.052050
2,2,20379.276748
3,3,19199.944000
4,4,17128.426080


In [57]:
children_amount_interquartile.head()

,children,charges
0,0,9856.95190
1,1,8483.87015
2,2,9264.97915
3,3,10600.54830
4,4,11033.66170


In [25]:
# Add a new column of weight status according to a patient's BMI
bmi_bins = [1, 16.4, 18.4, 24.9, 29.9, 34.9, 39.9, 100]
bmi_classes = ['Severely underweight', 'Underweight', 'Normal weight', 'Overweight', 'Obese class I', 'Obese class II', 'Obese class III']
insurance_records['bmi classification'] = pd.cut(insurance_records['bmi'], bins=bmi_bins, labels=bmi_classes)

# Obtain average insurance costs for patients in different BMI categories
average_cost_bmi = insurance_records.groupby('bmi classification').charges.mean().reset_index()
average_cost_bmi.head()

,bmi classification,charges
0,Severely underweight,1694.796400
1,Underweight,9228.906068
2,Normal weight,10379.499732
3,Overweight,11006.809989
4,Obese class I,14217.616975


In [42]:
# Find the first and third quartile and the interquartile range of insurance costs for different BMI classifications
bmi_first_q = insurance_records.groupby('bmi classification').charges.apply(lambda x: np.percentile(x, 25)).reset_index()
bmi_third_q = insurance_records.groupby('bmi classification').charges.apply(lambda x: np.percentile(x, 75)).reset_index()
bmi_interquartile = insurance_records.groupby('bmi classification').charges.apply(lambda x: np.percentile(x, 50)).reset_index()

In [43]:
bmi_first_q.head()

,bmi classification,charges
0,Severely underweight,1694.796400
1,Underweight,2971.324000
2,Normal weight,4057.701137
3,Overweight,4731.707250
4,Obese class I,4738.268200


In [44]:
bmi_third_q.head()

,bmi classification,charges
0,Severely underweight,1694.796400
1,Underweight,13016.870375
2,Normal weight,15197.264325
3,Overweight,15595.840620
4,Obese class I,16455.707850


In [45]:
bmi_interquartile.head()

,bmi classification,charges
0,Severely underweight,1694.796400
1,Underweight,6877.980100
2,Normal weight,8604.153525
3,Overweight,8659.378000
4,Obese class I,9411.005000


In [93]:
# Impacts of having children on one's BMI, smoking habit and living area
average_bmi_have_children = insurance_records.groupby('have children').bmi.mean().reset_index()
smoking_habit_children_amount = insurance_records.groupby('children').smoker.count().reset_index()
regions_have_children = insurance_records.groupby('region')['have children'].value_counts().reset_index()

In [60]:
average_bmi_have_children.head()

,have children,bmi
0,no,30.550296
1,yes,30.748370


In [74]:
smoking_habit_children_amount.head()

,children,smoker
0,0,574
1,1,324
2,2,240
3,3,157
4,4,25


In [94]:
regions_have_children.head(10)

,region,have children,count
0,northeast,yes,177
1,northeast,no,147
2,northwest,yes,193
3,northwest,no,132
4,southeast,yes,207
5,southeast,no,157
6,southwest,yes,187
7,southwest,no,138
